<center>
    <h1>EXPERIMENT</h1>
    <hr>
</center>

In [2]:
import os

from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

import ray
from ray import tune
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.integration.keras import TuneReportCallback
from ray.tune import JupyterNotebookReporter

from module.c_data_loader import data_load
from module.c_model import get_model
from module.c_test import get_test_model_config
from module.common import set_device, make_tag

def experiment(config):
    # prepare data
    trnx, trny, tstx, tsty = data_load(config['t_path'], 
                                       config['exp']['d_id'], 
                                       config['exp']['s_id'], 
                                       config['exp']['e_id'], 
                                       config['exp']['case'], 
                                       config['exp']['n_fold'], 
                                       config['model']['s_len'])

    trnx = trnx.reshape(-1, config['model']['s_len'], config['model']['n_dim'])
    tstx = tstx.reshape(-1, config['model']['s_len'], config['model']['n_dim'])
    
    # build model
    set_device(config['n_device'])
    model = get_model(config['exp']['m_id'], config['model'])

    # fit model
    m_c_path = f'{config["t_path"]}/model_checkpoint/{make_tag(config["exp"])}'    # model checkpoint path
    if os.path.exists(m_c_path) == False:
        os.mkdir(m_c_path)
        
    callbacks = [ReduceLROnPlateau(patience=10, mode='min'),
                 EarlyStopping(patience=10, mode='min'),
                 ModelCheckpoint(f'{m_c_path}/{make_tag(config["model"])}.h5', 
                                 save_best_only=True, mode='auto'),
                 TuneReportCallback({'mse': 'loss', 
                                     'mae': 'mae', 
                                     'r2': 'r2_score', 
                                     'val_mse': 'val_loss',  
                                     'val_mae': 'val_mae', 
                                     'val_r2': 'val_r2_score',
                                    })]

    model.fit(trnx, trny, 
              validation_split=0.1, shuffle=True, 
              batch_size=256, epochs=1, 
              callbacks=callbacks, 
              verbose=0
             )

def tuning(case, n_fold):
    # example
    config = {
        't_path': '/workspace/junwon/bp/_experiment',    # total absolute path
        'n_device': 3,                                   # device number
        'exp': {
            'd_id': 'd00',                               # dataset id
            's_id': 's01',                               # detail id
            'm_id': 'm01',                               # model id
            'e_id': 'e06',                               # experiment id
            'case': case,
            'n_fold': n_fold                             # fold number
        },

        'model': {
            's_len': 100,                                # sequence length
            'n_dim': 1,                                  # data dimension
            'n_unit': tune.randint(100, 200),            # of lstm unit
            'drop_rate': tune.uniform(0.1, 0.5),         # drop out rate
            'n_output': 2                                # # of output
        }
    }
    
    sched = AsyncHyperBandScheduler(time_attr="training_iteration", 
                                    max_t=400, grace_period=20)

    analysis = tune.run(
        experiment,
        name=make_tag(config["exp"]),
        scheduler=sched,
        metric="r2",
        mode="max",
        num_samples=5,
        resources_per_trial={
            'cpu': 1, 
            'gpu': 0.2
        },
        config=config,
        local_dir='./ray_results',
        progress_reporter=JupyterNotebookReporter(overwrite=True)
    )

In [10]:
case = 'case01'
n_fold = 3
ray.shutdown()
ray.init(num_gpus=1)
analysis = tuning(case, n_fold)

Trial name,status,loc,model/drop_rate,model/n_unit,iter,total time (s),mse,mae,r2
experiment_ee4ec_00000,TERMINATED,172.17.0.7:3375,0.486318,190,1,8.76092,5712.28,71.5846,-8.699
experiment_ee4ec_00001,TERMINATED,172.17.0.7:3400,0.182617,145,1,10.5162,5714.32,71.5986,-8.70913
experiment_ee4ec_00002,TERMINATED,172.17.0.7:3386,0.36721,188,1,10.797,5712.25,71.5842,-8.70233
experiment_ee4ec_00003,TERMINATED,172.17.0.7:3377,0.352551,174,1,10.1904,5713.5,71.5939,-8.70983
experiment_ee4ec_00004,TERMINATED,172.17.0.7:3397,0.485323,182,1,11.0233,5713.1,71.5921,-8.70758


2022-01-31 05:30:57,611	INFO tune.py:630 -- Total run time: 15.76 seconds (15.57 seconds for the tuning loop).
